In [1]:
#Importing libraries
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import label_binarize
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import sagemaker
import boto3

from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session



In [2]:
#Set the name for s3 bucket, where you will store all the files and the model.

bucket_name = 'spamclassifierbucket'

#Get the current AWS Region where the SageMaker notebook instance is running.
my_region = boto3.session.Session().region_name
print(my_region)


us-east-1


In [3]:
#Create S3 Bucket for the sagemaker notebook instance in the specific region where it is running
s3 = boto3.resource('s3')

try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print("s3 bucket created successfully")

except Exception as e:
    print("s3 bucket not created. Error: ", e)




s3 bucket created successfully


In [4]:
#Set a folder path to store the data and the models
s3_prefix = "spam-files"
s3_outpath = 's3://{}/{}/output'.format(bucket_name,s3_prefix)

print(s3_outpath)

s3://spamclassifierbucket/spam-files/output


In [5]:
#Upload the data or text file and store it in S3 bucket

data = pd.read_csv('SMSSpamCollection.txt', sep="\t", header=None, names = ['labels', 'messages'])

data.head()



,labels,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
# We need to pre-process the dataset 
#1. Convert the labels into numerical values 0's and 1's
#2. Use TF-IDF pipeline to pre-process the data

data['labels'] = label_binarize(data['labels'],classes = ['ham','spam'])

#convert messages into tf_idf
tfidf = Pipeline([('cv',CountVectorizer()), ('tfidf_transformer', TfidfTransformer(smooth_idf=True,use_idf=True))])
tfidf_vector = pd.DataFrame(tfidf.fit_transform(data['messages']).todense())


In [7]:
#Split the data into train, validation and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(tfidf_vector, data['labels'], test_size=0.3, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.3, random_state=42)


In [18]:
import os
from sagemaker.session import s3_input, Session

# We are concatenating the X_train, Y_train in such a way that target label is the first column and then converting 
# it into a csv file.
pd.concat([Y_train, X_train], axis=1).to_csv(os.path.join('train.csv'), header=False, index=False)

# Next, we upload this csv file into the s3 bucket which we created earlier. This file will be under a separate folder
# called train folder within the s3 bucket.
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(s3_prefix, 'train/train.csv')).upload_file('train.csv')

# Use the TrainingInput class to configure a data input flow for training by using the uploaded training dataset. 
# This will be used during model training
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, s3_prefix), content_type='csv')

In [19]:
#Similar to previous step we follow the same for the test data
#Create csv file for test data, create a folder path within the s3 bucket for test and upload the file in that folder.


pd.concat([Y_test, X_test], axis=1).to_csv(os.path.join('test.csv'), header=False, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(s3_prefix, 'test/test.csv')).upload_file('test.csv')

# Use the TrainingInput class to configure a data input flow for testing by using the uploaded testing dataset. 
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, s3_prefix), content_type='csv')

In [20]:
# #Create csv file for validation data, create a folder path within the s3 bucket for validation and upload the file in that folder.

pd.concat([Y_val, X_val], axis=1).to_csv(os.path.join('val.csv'), header=False, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(s3_prefix, 'val/val.csv')).upload_file('val.csv')

# Use the TrainingInput class to configure a data input flow for validation by using the uploaded validation dataset. 
# Will used during model training
s3_input_val = sagemaker.TrainingInput(s3_data='s3://{}/{}/val'.format(bucket_name, s3_prefix), content_type='csv')

In [22]:
# Build a model using XGBoost inbuilt algorithm
# Use the xgboost algorithm image and run it to create an xgboost container

container = get_image_uri(boto3.Session().region_name,'xgboost',repo_version='1.2-1')

hyperparameters ={
"max_depth": 5,
"eta": 0.2,
"gamma": 2,
"min_child_weight": 5,
"subsample": 0.8,
"objective": "binary:logistic",
"early_stopping_rounds": 25,
"num_round": 150,
}

#constructing a SageMaker estimator for training that calls the xgboost-container.

#Estimators are a high-level interface for SageMaker training to handle end-to-end Amazon SageMaker training 
#and deployment tasks. 

estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m4.xlarge', 
                                          output_path=s3_outpath,
                                          sagemaker_session = sagemaker.Session())

#image_uri – Specify the training container image URI.

#role – The AWS Identity and Access Management (IAM) role that SageMaker uses to perform tasks on your behalf.

#train_instance_count and train_instance_type – The type and number of Amazon EC2 ML compute instances to use for training. 
#                                               In this we use a single ml.m4.xlarge instance,
#                                               which has 4 CPUs, 16 GB of memory, an Amazon Elastic Block Store 
#                                               (Amazon EBS) storage, and a high network performance. 

#train_volume_size – The size, in GB, of the EBS storage volume to attach to the training instance.

#sagemaker_session – The session object that manages interactions with SageMaker API operations 
#                    and other AWS service that the training job uses.

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [23]:
estimator.fit({"train":s3_input_train, "validation":s3_input_val})

2021-03-08 23:19:36 Starting - Starting the training job...
2021-03-08 23:20:00 Starting - Launching requested ML instancesProfilerReport-1615245575: InProgress
......
2021-03-08 23:21:01 Starting - Preparing the instances for training.........
2021-03-08 23:22:24 Downloading - Downloading input data
2021-03-08 23:22:24 Training - Downloading the training image.....[2021-03-08 23:23:14.509 ip-10-2-167-168.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INF

In [ ]:
# Deploy the model

# 1. Deploy the Model to SageMaker Hosting Services, Amazon EC2
# 2. Use SageMaker Predictor to Reuse the Hosted Endpoint
# 3. Make Prediction with Batch Transform

In [25]:
#Let's try 3rd option first since we have a test file already to make batch predictions
# Instead of hosting an endpoint in production, we can run one-time batch inference job to make predictions on a test data.

# In this option, we use the Sagemaker's Batch Transform functionality to test the model.
# The test data is split into batches and fed to the model. 
# The predictions from each batch is later merged to determine the model accuracy.

#create a transformer object

xgb_transformer = estimator.transformer(instance_count=1, instance_type='ml.m4.xlarge')




In [26]:
# Initiate the batch transform job by executing the transform() method of the xgb_transformer object.

# Specify S3 bucket URIs of input and output for the batch transform job 
X_test.to_csv(os.path.join('test_data.csv'), header=False, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(s3_prefix, 'batchtest/test_data.csv')).upload_file('test_data.csv')
batch_input = 's3://{}/{}/batchtest'.format(bucket_name, s3_prefix)
batch_output = 's3://{}/{}/batch-prediction'.format(bucket_name, s3_prefix)


xgb_transformer.transform(data=batch_input , data_type='S3Prefix' , content_type='text/csv', split_type='Line')


........................................[2021-03-09:00:18:04:INFO] No GPUs detected (normal if no gpus installed)
[2021-03-09:00:18:04:INFO] No GPUs detected (normal if no gpus installed)
[2021-03-09:00:18:04:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;

  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }

  server {
    listen 8080 deferred;
    client_max_body_size 0;

    keepalive_timeout 3;

    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }

    location / {
      return 404 "{}";
    }

  }
}


2021/03/09 00:18:04 [crit] 19#19:

In [27]:
xgb_transformer.wait()

[2021-03-09:00:18:04:INFO] No GPUs detected (normal if no gpus installed)
[2021-03-09:00:18:04:INFO] No GPUs detected (normal if no gpus installed)
[2021-03-09:00:18:04:INFO] nginx config: 
worker_processes auto;
[2021-03-09:00:18:04:INFO] No GPUs detected (normal if no gpus installed)
[2021-03-09:00:18:04:INFO] No GPUs detected (normal if no gpus installed)
[2021-03-09:00:18:04:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;

  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }

  server {
    listen 8080 deferred;
    client_max_body_size 0;

    keepalive_timeout 3;

    location ~ ^/(ping|invocations|execution-parameter

In [30]:
#When the batch transform job is complete, SageMaker creates the test.csv.out prediction data saved in the batch_output path
#Download the test.csv.out to current directory
!aws s3 cp --recursive $xgb_transformer.output_path $batch_output



copy: s3://sagemaker-us-east-1-124958237186/sagemaker-xgboost-2021-03-09-00-11-32-487/test_data.csv.out to s3://spamclassifierbucket/spam-files/batch-prediction/test_data.csv.out


In [37]:
try:
    predictions = pd.read_csv(os.path.join('s3://spamclassifierbucket/spam-files/batch-prediction/', 'test_data.csv.out'), header=None)
    predictions = [round(num) for num in predictions.squeeze().values]
except Exception as e:
    print("error:", e)

In [40]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

accuracy_score(Y_test, predictions)



0.9712918660287081

In [44]:
print(confusion_matrix(Y_test, predictions))

[[1439    9]
 [  39  185]]


In [45]:
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1448
           1       0.95      0.83      0.89       224

    accuracy                           0.97      1672
   macro avg       0.96      0.91      0.93      1672
weighted avg       0.97      0.97      0.97      1672



In [54]:
# Clean up 
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)


The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [55]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'V0HAV84HJD1585EF',
   'HostId': 'pGg24RSY4vWe+yMvNtlP8dXuQz6adMYDmmhQF7nz7FP52JCD0rkRRm6ri5CgEKkH+mmupj8c2y8=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'pGg24RSY4vWe+yMvNtlP8dXuQz6adMYDmmhQF7nz7FP52JCD0rkRRm6ri5CgEKkH+mmupj8c2y8=',
    'x-amz-request-id': 'V0HAV84HJD1585EF',
    'date': 'Tue, 09 Mar 2021 01:17:55 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'spam-files/output/sagemaker-xgboost-2021-03-08-23-19-35-778/rule-output/ProfilerReport-1615245575/profiler-output/profiler-reports/BatchSize.json'},
   {'Key': 'spam-files/batchtest/test_data.csv'},
   {'Key': 'spam-files/output/sagemaker-xgboost-2021-03-08-23-19-35-778/profiler-output/system/incremental/2021030823/1615245780.algo-1.json'},
   {'Key': 'spam-files/output/sagemaker-xgboost-2021-03-08-23-19-35-778/output/model.tar.gz'},
   {'Key'